In [1]:
import os

ruta_base = '../../'

# Cambiar el directorio de trabajo
os.chdir(ruta_base)

print("Directorio de trabajo actual:", os.getcwd())

Directorio de trabajo actual: /home/david/proyectos4/repositorios/Proyectos-SGBA1/data


In [2]:
import pandas as pd
df_clima_ejemplo = pd.read_csv('processed/datos_clima/0200E_clima_completo.csv')

In [3]:
# Tipos de datos de las columnas
print(df_clima_ejemplo.dtypes)

fecha           object
indicativo      object
nombre          object
provincia       object
altitud          int64
tmed            object
prec            object
tmin            object
horatmin        object
tmax            object
horatmax        object
dir            float64
velmedia        object
racha           object
horaracha       object
sol             object
presMax         object
horaPresMax     object
presMin         object
horaPresMin     object
hrMedia        float64
dtype: object


# Numerizar datos

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from datetime import datetime

In [5]:
df_procesado = df_clima_ejemplo.copy()

### Procesar fechas

In [6]:
# 1. Procesamiento de fechas y horas
# Convertir fecha a datetime y extraer características
df_procesado['fecha'] = pd.to_datetime(df_procesado['fecha'])
df_procesado['año'] = df_procesado['fecha'].dt.year
df_procesado['mes'] = df_procesado['fecha'].dt.month
df_procesado['dia'] = df_procesado['fecha'].dt.day
df_procesado['dia_semana'] = df_procesado['fecha'].dt.dayofweek
df_procesado['estacion_año'] = df_procesado['fecha'].dt.month % 12 // 3

### Procesar columnas de Horas y minutos

In [7]:
print(df_procesado.columns)

Index(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec',
       'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha',
       'horaracha', 'sol', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin',
       'hrMedia', 'año', 'mes', 'dia', 'dia_semana', 'estacion_año'],
      dtype='object')


In [8]:
print(df_procesado['horaPresMax'].unique())
print(df_procesado['horaPresMin'].unique())

['00' '24' 'Varias' '22' '11' '08' '23' '02' '20' '03' '10' '09' '19' '12'
 '21' '05' '01' '13' '07' '06' '14']
['24' '05' '00' '06' 'Varias' '03' '17' '14' '16' '04' '15' '01' '21' '12'
 '02' '22' '18' '23' '19' '20' '08' '09' '11' '13' '10' '07']


In [9]:
print(df_procesado['horaracha'].unique())
print(df_procesado['horatmax'].unique())
print(df_procesado['horatmin'].unique())

['23:02' '01:02' '00:12' ... '22:56' '14:32' '13:15']
['11:20' '01:39' '13:43' '14:25' 'Varias' '15:35' '04:05' '14:55' '09:34'
 '13:54' '13:40' '00:13' '12:28' '13:55' '13:10' '12:32' '14:35' '14:04'
 '13:05' '10:18' '15:10' '12:36' '12:07' '13:42' '13:38' '14:26' '10:27'
 '11:52' '13:13' '14:30' '14:16' '14:05' '14:40' '14:32' '12:53' '11:07'
 '00:00' '13:41' '12:47' '13:22' '12:45' '12:52' '12:58' '13:01' '14:11'
 '14:20' '13:47' '14:47' '12:19' '15:13' '14:29' '14:45' '14:12' '14:50'
 '14:52' '13:25' '13:21' '13:03' '13:50' '10:28' '14:15' '11:00' '12:17'
 '15:09' '13:28' '12:18' '12:57' '23:59' '00:55' '13:52' '13:15' '16:10'
 '01:15' '14:48' '13:49' '13:19' '13:20' '16:02' '15:00' '14:31' '11:24'
 '13:16' '12:11' '16:30' '15:15' '14:07' '14:13' '12:30' '03:29' '15:31'
 '12:10' '11:13' '12:29' '13:30' '12:35' '09:24' '12:05' '10:55' '10:58'
 '11:40' '11:17' '12:40' '13:44' '12:50' '13:35' '12:41' '12:20' '12:25'
 '12:38' '12:08' '10:15' '11:29' '11:41' '14:09' '11:30' '12:04' '15:

In [10]:
for col in ['horatmin', 'horatmax', 'horaracha', ]:
    if col in df_procesado.columns:
        # Ensure the column is of string type before applying the ~ operator
        df_procesado[col] = df_procesado[col].astype(str)
        print(col)
        print(df_procesado[col][~df_procesado[col].str.contains(':')].unique())
    else:
        print(f"La columna '{col}' no existe en el DataFrame.")

horatmin
['Varias']
horatmax
['Varias' 'nan']
horaracha
['nan']


In [11]:
def convertir_hora_a_minutos(x):
    if pd.isna(x) or x == 'nan':
        return pd.NA
    
    if x == 'Varias':
        # -1 podría ser un marcador para 'Varias'
        # (lo normalizaremos después, así que el -1 no afectará al modelo)
        return -1
        
    # Solo proceder si tiene el formato HH:MM esperado
    if isinstance(x, str) and ':' in x and x.replace(':', '').isdigit():
        try:
            horas, minutos = x.split(':')
            return int(horas) * 60 + int(minutos)
        except (ValueError, TypeError):
            return pd.NA
    else:
        return pd.NA

In [12]:
# Convertir horas a valores numéricos con codificación especial
max_minutes = 24 * 60  # Total minutos en un día

for col in ['horatmin', 'horatmax', 'horaracha']:
    if col in df_procesado.columns:
        # Crear columna para indicar si es 'Varias'
        df_procesado[f'{col}_varias'] = (df_procesado[col] == 'Varias').astype(int)
        df_procesado[f'{col}_minutos'] = df_procesado[col].apply(convertir_hora_a_minutos)
        
        # Paso adicional: Normalizar entre 0 y 2pi para representación cíclica del tiempo
        # Filtrar solo los valores válidos (que no son NaN o -1)
        valid_mask = df_procesado[f'{col}_minutos'].notna() & (df_procesado[f'{col}_minutos'] != -1)
        valid_minutes = df_procesado.loc[valid_mask, f'{col}_minutos']
        
        # Crear las columnas seno y coseno para capturar la naturaleza cíclica del tiempo
        df_procesado[f'{col}_sin'] = np.nan
        df_procesado[f'{col}_cos'] = np.nan
        
        # Asignar los valores trigonométricos solo para los valores válidos
        if not valid_minutes.empty:
            # Convertir minutos a ángulos (0 a 2π)
            angles = valid_minutes.astype(float) * (2 * np.pi / max_minutes)
            df_procesado.loc[valid_mask, f'{col}_sin'] = np.sin(angles)
            df_procesado.loc[valid_mask, f'{col}_cos'] = np.cos(angles)

# Comprobar que las columnas se han creado correctamente
print("Columnas para horatmin:")
print(df_procesado[['horatmin_varias', 'horatmin_sin', 'horatmin_cos']].head())
print("\nColumnas para horatmax:")
print(df_procesado[['horatmax_varias', 'horatmax_sin', 'horatmax_cos']].head())
print("\nColumnas para horaracha:")
print(df_procesado[['horaracha_varias', 'horaracha_sin', 'horaracha_cos']].head())
        

Columnas para horatmin:
   horatmin_varias  horatmin_sin  horatmin_cos
0                0      0.982450      0.186524
1                0     -0.004363      0.999990
2                0      0.896873      0.442289
3                0      0.965926      0.258819
4                0      0.884988      0.465615

Columnas para horatmax:
   horatmax_varias  horatmax_sin  horatmax_cos
0                0      0.173648     -0.984808
1                0      0.418660      0.908143
2                0     -0.434445     -0.900698
3                0     -0.591310     -0.806445
4                1           NaN           NaN

Columnas para horaracha:
   horaracha_varias  horaracha_sin  horaracha_cos
0                 0      -0.250380       0.968148
1                 0       0.267238       0.963630
2                 0       0.052336       0.998630
3                 0       0.719340       0.694658
4                 0       0.771625       0.636078


### Procesar columnas de solo Horas

In [13]:
def convertir_hora_entera(x):
    if pd.isna(x) or x == 'nan':
        return pd.NA
    
    if x == 'Varias':
        return -1
    
    try:
        # Convertir a entero (estas columnas solo tienen horas enteras)
        hora = int(x)
    
        # Manejar el caso especial de '24' (debería ser '00')
        if hora == 24:
            hora = 0
            
        # Convertir a minutos para mantener consistencia con otras columnas
        return hora * 60
    except (ValueError, TypeError):
        return pd.NA

In [14]:
# Convertir horas de presión a valores numéricos (formato específico)
for col in ['horaPresMax', 'horaPresMin']:
    if col in df_procesado.columns:
        # Crear columna para indicar si es 'Varias'
        df_procesado[f'{col}_varias'] = (df_procesado[col] == 'Varias').astype(int)
        df_procesado[f'{col}_minutos'] = df_procesado[col].apply(convertir_hora_entera)

        # Normalización circular (valores entre 0 y 2π)
        valid_minutes = df_procesado[df_procesado[f'{col}_minutos'] >= 0][f'{col}_minutos']
        if not valid_minutes.empty:
            max_minutes = 24 * 60  # 24 horas * 60 minutos
            
            # Convertir a valores circulares
            df_procesado[f'{col}_sin'] = np.where(
                df_procesado[f'{col}_minutos'] >= 0,
                np.sin(2 * np.pi * df_procesado[f'{col}_minutos'] / max_minutes),
                0  # Valor por defecto para NA o 'Varias'
            )
            
            df_procesado[f'{col}_cos'] = np.where(
                df_procesado[f'{col}_minutos'] >= 0,
                np.cos(2 * np.pi * df_procesado[f'{col}_minutos'] / max_minutes),
                0  # Valor por defecto para NA o 'Varias'
            )
        
        # Eliminar la columna original y la intermedia
        df_procesado = df_procesado.drop(columns=[col, f'{col}_minutos'])

# Comprobar que las columnas se han creado correctamente
print("Columnas para horaPresMax:")
print(df_procesado[['horaPresMax_varias', 'horaPresMax_sin', 'horaPresMax_cos']].tail())
print("\nColumnas para horaPresMin:")
print(df_procesado[['horaPresMin_varias', 'horaPresMin_sin', 'horaPresMin_cos']].head())

Columnas para horaPresMax:
      horaPresMax_varias  horaPresMax_sin  horaPresMax_cos
2515                   0         0.000000         1.000000
2516                   0         0.258819        -0.965926
2517                   0         0.258819         0.965926
2518                   1         0.000000         0.000000
2519                   1         0.000000         0.000000

Columnas para horaPresMin:
   horaPresMin_varias  horaPresMin_sin  horaPresMin_cos
0                   0         0.000000     1.000000e+00
1                   0         0.965926     2.588190e-01
2                   0         0.000000     1.000000e+00
3                   0         0.000000     1.000000e+00
4                   0         1.000000     6.123234e-17


### Procesar precipitación

In [15]:
# Valores únicos en la columna 'precipitacion'
print(df_procesado['prec'].unique())

['20,6' '8,9' 'Ip' '0,0' '0,1' '0,8' '6,1' '1,1' '1,2' '0,7' '19,2' '10,8'
 '1,3' '35,3' '26,1' '4,0' '3,0' '0,4' '2,7' '12,5' '0,5' '5,3' '7,6'
 '9,6' '9,1' '8,5' '13,8' '4,5' '3,9' '4,1' '1,6' '0,3' '5,4' '47,0' '6,2'
 '4,7' '4,2' '1,7' '5,2' '2,5' '4,8' '13,6' '8,0' '4,4' '0,9' '7,9' '2,1'
 '19,7' '2,4' '0,2' '23,5' '6,4' '11,3' '3,1' '1,5' '1,9' '1,4' '6,9'
 '5,5' '15,0' '8,7' '1,0' '12,8' '9,2' '10,0' '2,9' '9,5' '12,2' '3,7'
 '3,4' '8,6' '16,3' '12,1' '11,9' '2,0' '0,6' '26,7' '66,0' '2,8' '31,7'
 '16,7' '6,6' '21,9' '12,6' '12,0' '7,4' '20,2' '16,1' '2,3' '10,3' '19,3'
 '115,1' '7,0' '22,4' '7,7' '15,1' '19,6' '3,5' '5,8' '3,8' '2,2' '14,9'
 '13,3' '7,1' '12,7' '50,0' '75,7' '7,3' '30,8' '8,8' '33,1' '26,3' '16,6'
 '38,8' '46,6' '31,1' '6,0' '6,8' '24,1' '7,8' '2,6' '14,7' '6,7' '13,9'
 '36,4' '10,4' '3,6' '13,4' '16,5' '5,1' '20,3' '29,4' '27,2' '34,0' '7,5'
 '23,6' '21,4' '32,2' '5,9' '23,9' '96,8' '44,2' '10,9' '4,3' '9,0' '28,8'
 '35,0' '27,9' '1,8' '17,5' '3,2' '52,5' '12,9

In [16]:
def convertir_precipitacion(x):
        if pd.isna(x):
            return np.nan
        
        if isinstance(x, (int, float)):
            return float(x)
        
        if isinstance(x, str) and x == 'Ip':
            # Valor inapreciable: asignamos un valor pequeño (0.05 mm)
            return 0.05
            
        # Por si hay algún otro string inesperado
        try:
            return float(x)
        except (ValueError, TypeError):
            return np.nan

In [17]:
# Procesamiento simplificado de la columna de precipitación
if 'prec' in df_procesado.columns:
    # Crear columna indicadora para precipitación inapreciable
    df_procesado['prec_inapreciable'] = df_procesado['prec'].apply(
        lambda x: 1 if isinstance(x, str) and x == 'Ip' else 0)
       
    # Aplicar la conversión
    df_procesado['prec_valor'] = df_procesado['prec'].apply(convertir_precipitacion)
    
    # Transformación logarítmica para manejar la naturaleza sesgada de las precipitaciones
    # Agregamos 1 para evitar log(0) y usar log(1+x)
    df_procesado['prec_log'] = np.log1p(df_procesado['prec_valor'])
    
    # Eliminamos la columna original
    df_procesado = df_procesado.drop(columns=['prec'])

# Comprobar que las columnas se han creado correctamente
print("Columnas para precipitación:")
print(df_procesado[['prec_inapreciable', 'prec_valor', 'prec_log']].head())

Columnas para precipitación:
   prec_inapreciable  prec_valor  prec_log
0                  0         NaN       NaN
1                  0         NaN       NaN
2                  1        0.05   0.04879
3                  0         NaN       NaN
4                  1        0.05   0.04879


### Asegurar que las columnas float lo son

In [18]:
# Asegurar que las columnas que deben ser de tipo float lo sean
# altitud, tmed, tmin, tmax, dir, velmedia, sol, presmax, presmin, hrmedia

# Verificar qué columnas existen en el dataframe
available_cols = []
for col in ['altitud', 'tmed', 'tmin', 'tmax', 'dir', 'velmedia', 'racha', 'sol', 'presMax', 'presMin', 'hrMedia']:
	if col in df_procesado.columns:
		available_cols.append(col)

# Convertir columnas a float si existen en el dataframe
for col in available_cols:
	try:
		df_procesado[col] = df_procesado[col].astype(float)
	except ValueError:
		# Si hay valores que no se pueden convertir (como comas en lugar de puntos)
		if isinstance(df_procesado[col][0], str) and ',' in df_procesado[col][0]:
			df_procesado[col] = df_procesado[col].str.replace(',', '.').astype(float)
		else:
			print(f"No se pudo convertir la columna {col} a tipo float.")

# Comprobar que las columnas se han convertido correctamente
print(df_procesado[available_cols].dtypes)

altitud     float64
tmed        float64
tmin        float64
tmax        float64
dir         float64
velmedia    float64
racha       float64
sol         float64
presMax     float64
presMin     float64
hrMedia     float64
dtype: object


## Verificación de tipos de datos

In [19]:
print(df_procesado.dtypes)


fecha                 datetime64[ns]
indicativo                    object
nombre                        object
provincia                     object
altitud                      float64
tmed                         float64
tmin                         float64
horatmin                      object
tmax                         float64
horatmax                      object
dir                          float64
velmedia                     float64
racha                        float64
horaracha                     object
sol                          float64
presMax                      float64
presMin                      float64
hrMedia                      float64
año                            int32
mes                            int32
dia                            int32
dia_semana                     int32
estacion_año                   int32
horatmin_varias                int64
horatmin_minutos               int64
horatmin_sin                 float64
horatmin_cos                 float64
h

In [20]:
print(df_procesado['racha'].unique())

[17.8 18.9 11.4 10.3 10.8  8.6 11.9  9.7 17.2 14.4 14.2  5.8  4.7  7.5
 12.2 10.6 10.  15.  16.4 21.7 12.8 11.7 15.3 15.6 25.3 16.1 13.6 17.5
 13.3 15.8  4.2 14.7 16.9 13.1  9.4 23.1 18.1  9.2  6.4  8.9 18.3  6.1
 12.5 21.9  7.2  8.1 21.1  7.8  6.9  6.7  5.3  4.4 19.2 19.7  nan 21.4
 22.8 20.6 20.  18.6 24.4 26.1 24.2 20.3 20.8 25.6 23.3 24.7 28.6 28.9
 11.1  8.3 13.9]


In [21]:
# Eliminar columnas que no se usarán en el modelo

# nombre                        object
# provincia                     object
# horatmin                      object
# horatmax                      object
# horaracha                     object

columnas_eliminar = ['nombre', 'provincia', 'horatmin', 'horatmax', 'horaracha']
df_procesado = df_procesado.drop(columns=columnas_eliminar)

print(df_procesado.columns)

Index(['fecha', 'indicativo', 'altitud', 'tmed', 'tmin', 'tmax', 'dir',
       'velmedia', 'racha', 'sol', 'presMax', 'presMin', 'hrMedia', 'año',
       'mes', 'dia', 'dia_semana', 'estacion_año', 'horatmin_varias',
       'horatmin_minutos', 'horatmin_sin', 'horatmin_cos', 'horatmax_varias',
       'horatmax_minutos', 'horatmax_sin', 'horatmax_cos', 'horaracha_varias',
       'horaracha_minutos', 'horaracha_sin', 'horaracha_cos',
       'horaPresMax_varias', 'horaPresMax_sin', 'horaPresMax_cos',
       'horaPresMin_varias', 'horaPresMin_sin', 'horaPresMin_cos',
       'prec_inapreciable', 'prec_valor', 'prec_log'],
      dtype='object')


### Guardar el dataframe

In [22]:
!pwd

/home/david/proyectos4/repositorios/Proyectos-SGBA1/data


In [26]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [27]:
# Guardar el dataframe procesado
ruta = 'processed/datos_clima/0200E_clima_numerizado'
df_procesado.to_parquet(ruta+".parquet", index=False)
df_procesado.to_csv(ruta+".csv", index=False)

